# 15 · A Confirmação nas Alturas Estratosféricas

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números Primos — Investigações Computacionais sob o Espelho de Euler*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais. Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação. Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.


---

## 1. O Contexto Histórico: Montgomery, Berry e Odlyzko

Para compreender a magnitude do que a análise final deste notebook revela, é essencial situar o problema no seu contexto histórico. A conexão profunda entre números primos e caos quântico não surgiu como um acaso isolado, mas como o resultado de uma das mais notáveis convergências entre matemática e física do século XX.

A jornada desenvolvida neste notebook ecoa, em escala didática e computacional, essa mesma aventura intelectual.

#### A faísca inicial: Montgomery e Dyson (anos 1970)

O ponto de partida foi um encontro fortuito. Hugh Montgomery, ao estudar a correlação entre os zeros não triviais da função zeta de Riemann — frequentemente descritos como os “primos” da análise complexa — apresentou seus resultados a Freeman Dyson. A reação foi imediata: Dyson reconheceu que a fórmula obtida por Montgomery coincidia exatamente com a correlação entre níveis de energia em núcleos atômicos pesados, um domínio descrito pela Teoria  
de Matrizes Aleatórias (RMT).

Pela primeira vez, uma ponte concreta ligava teoria dos números e física quântica.

#### O enquadramento físico: Michael Berry (anos 1980)

Se a conexão existia, faltava compreender o porquê. Michael Berry forneceu esse enquadramento ao estabelecer a **Teoria de Matrizes Aleatórias (RMT)** como a linguagem universal do **caos quântico**.

Sua contribuição, consolidada na conjectura de Bohigas–Giannoni–Schmit (BGS), afirma que sistemas quânticos cujo análogo clássico é caótico apresentam espectros descritos por estatísticas da RMT no regime adequado. A classe específica dessa estatística — GOE ou GUE — é determinada pelas simetrias fundamentais do sistema, como a presença ou ausência de reversão temporal.

Nesse contexto, os zeros da função zeta e a estrutura profunda dos primos passaram a ser interpretados como a assinatura espectral de um sistema quântico caótico abstrato. Enquanto os zeros no plano complexo manifestam a classe GUE (associada à quebra de simetria), a aritmética real sob o operador $M$ revela a classe GOE, protegida por sua própria reversibilidade estrutural.

#### A confirmação computacional: Andrew Odlyzko (anos 1980 e 1990)

A conjectura precisava ser testada em escalas extremas. Andrew Odlyzko realizou esse feito ao calcular trilhões de zeros da função zeta em alturas sem precedentes, como $10^{22}$ e além. Utilizando supercomputadores e algoritmos altamente otimizados, Odlyzko mostrou que os dados coincidiam de forma impressionante com as previsões da **GUE (*Gaussian Unitary Ensemble*)**.

Essa confirmação consolidou a conexão entre os primos e o caos quântico. O laboratório desenvolvido neste notebook presta homenagem a essa trajetória, mas com uma distinção fundamental: **enquanto os zeros no plano complexo revelam a rigidez GUE, nossa análise da reta aritmética real, ancorada na unidade, revela a emergência da classe GOE — a assinatura de sistemas que preservam a simetria de espelhamento**.

---

## 2. O Desafio Final: Dos Limites da Computação à Força da Teoria

Os capítulos anteriores estabeleceram um resultado robusto: a estatística GOE emerge de forma estável a partir de $X_0 \approx 10^5$.

A questão que permanece é inevitável: essa lei persiste nas **alturas estratosféricas** onde Odlyzko observou os zeros da função zeta, em escalas como $10^{22}$ ou maiores?

A resposta direta é computacionalmente inacessível. Nessas regiões, a contagem exata de primos torna-se impraticável, mesmo com os melhores algoritmos conhecidos.

Para superar essa limitação, recorremos a uma ponte teórica fundamental: a **função $R(x)$ de Riemann**, que fornece uma representação analítica assintótica extremamente precisa para $\pi(x)$, mesmo em domínios astronômicos. Essa construção permite sondar regiões da reta numérica que estariam fora do alcance de qualquer método elementar de contagem.

---

## 3. A Análise dos Dados Estratosféricos

Neste laboratório final, não geramos novos dados brutos. Em vez disso, analisamos um *dataframe* previamente gerado pelo próprio código deste projeto, correspondente a uma varredura de valores de $X_0$ desde $10^8$ até $10^{28}$.

Esses dados foram obtidos por meio de cálculos de alta precisão, nos quais a função $R(x)$ substitui explicitamente a contagem direta de primos.

O objetivo é acompanhar o comportamento de duas impressões digitais fundamentais do caos quântico:

- a média da razão de espaçamentos adjacentes, **$\langle r \rangle$**;
- o **Participation Ratio normalizado**, $PR/N$.

Ambas são analisadas como funções da altura $X_0$.

### Hipótese final

Se a conexão entre os primos e a estatística GOE for estrutural no regime observacional considerado, então tanto $\langle r \rangle$ quanto $PR/N$ devem permanecer próximos aos valores teóricos da GOE ($\langle r \rangle \approx 0.536$ e $PR/N \approx 0.333$), mesmo quando exploramos as fronteiras mais distantes da reta numérica.

### Reprodutibilidade

Este notebook **executa integralmente o experimento estratosférico** e permite a sua replicação direta, dentro dos limites computacionais descritos acima.  

Os dados não são simulados nem ajustados: a contagem explícita de primos é substituída pela função $R(x)$ de Riemann, seguindo o mesmo princípio epistemológico adotado por Odlyzko: substituir a inacessibilidade direta por um objeto analítico cuja precisão é controlada e verificável.


In [1]:
# Requisitos: mpmath, sympy, numpy, matplotlib, ipywidgets
# !pip install mpmath sympy

# Atenção: N >= 4096 em X0 >= 1e24 pode levar vários minutos,
# devido à avaliação de R(x) em alta precisão.

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time
import mpmath as mp

# --- CONFIGURAÇÃO DE ALTA PRECISÃO ---
mp.mp.dps = 200

# --- FUNÇÕES PARA A ANÁLISE ESTRATOSFÉRICA ---

def F_R_approx(x):
    """
    Aproxima Δπ(x) usando a função R de Riemann.
    Implementação explícita para evitar pseudo-vetorização lenta.
    """
    return np.array(
        [float(mp.riemannr(v) - 2 * mp.riemannr(v / 2)) for v in x],
        dtype=np.float64
    )

def generate_dual_phase_matrix(fx_values, x_values, a=1.0, b=np.sqrt(2.0)):
    """Gera a matriz M usando o Kernel Dual-Phase, robusto para altas escalas."""
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    
    Phi = np.outer(fx, logx)
    M = 0.5 * (np.cos(a * Phi) + np.cos(b * Phi.T))
    
    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev
        
    return 0.5 * (M + M.T)

def r_stat(lam, alpha=0.10):
    """
    Estatística <r> calculada no bulk do espectro.
    """
    lam_sorted = np.sort(lam)
    N = len(lam_sorted)
    k0, k1 = int(alpha * N), int((1 - alpha) * N)
    lam_bulk = lam_sorted[k0:k1]

    s = np.diff(lam_bulk)
    s = s[s > 0]
    if len(s) < 2:
        return np.nan

    r = np.minimum(s[1:], s[:-1]) / np.maximum(s[1:], s[:-1])
    return r.mean()

# --- FUNÇÃO ADICIONADA PARA ANÁLISE DOS AUTOVETORES ---
def participation_ratio(v_matrix):
    """Calcula o Participation Ratio para uma matriz de autovetores."""
    return 1 / np.sum(v_matrix**4, axis=0)

# --- A FUNÇÃO INTERATIVA PRINCIPAL (AJUSTADA) ---
# Nota:
# Para X₀ ≥ 10^24 e N ≥ 4096, a avaliação de R(x) em alta precisão
# pode levar vários minutos. Isso é esperado e documenta
# o custo computacional do regime estratosférico.

def stratospheric_lab_final(N=2048, log_X0=22.0, span=2.0):
    
    X0 = mp.power(10, log_X0)
    print(f"Iniciando análise em X₀ ≈ 10^{log_X0:.0f}...")
    
    # --- GERAÇÃO DA GRADE 'x' ---
    print("-> Gerando a grade 'x' com mpmath...")
    X0_mp, span_mp = mp.mpf(X0), mp.mpf(span)
    L0_mp = mp.log(X0_mp)
    x_mp = [mp.exp(L0_mp - span_mp/2 + i * span_mp/(N-1)) for i in range(N)]
    x_log = np.array([float(v) for v in x_mp])
    
    # --- ANÁLISE ---
    print("-> Calculando o sinal F(x) com a Função R de Riemann...")
    fx_log = F_R_approx(x_log)
    
    print("-> Construindo a matriz M com o kernel Dual-Phase...")
    M_log = generate_dual_phase_matrix(fx_log, x_log)
    
    print("-> Calculando autovalores e autovetores...")
    # Captura autovalores (lam_log) E autovetores (v_log)
    lam_log, v_log = np.linalg.eigh(M_log)
    
    # CÁLCULO DAS MÉTRICAS IMPORTANTES
    # Métrica 1: Espaçamento de autovalores <r>
    # Nota: <r> é uma estatística local, insensível ao unfolding global,
    # adequada para comparação com RMT em regimes espectrais controlados.
    
    r_mean = r_stat(lam_log)
    # Métrica 2: Participation Ratio dos autovetores PR/N
    pr_values = participation_ratio(v_log)
    pr_n_mean = np.mean(pr_values / N)

    # IMPRESSÃO DAS MÉTRICAS-CHAVE
    print("\n----------------------------------------------------")
    print("  MÉTRICAS DE COMPROVAÇÃO GOE")
    print("----------------------------------------------------")
    print(f"  Autovalores -> <r> médio:")
    print(f"    - Medido:    {r_mean:.4f}")
    print(f"    - Teórico (GOE): ~0.536")
    print("\n")
    print(f"  Autovetores -> PR/N médio:")
    print(f"    - Medido:    {pr_n_mean:.4f}")
    print(f"    - Teórico (GOE): ~0.333")
    print("----------------------------------------------------\n")

    print("-> Gerando gráficos...")
    s_log = np.diff(np.sort(lam_log)); s_log = s_log[s_log > 0]
    s_unfolded_log = s_log / s_log.mean()
    
    # PLOT LADO A LADO (1x2)
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    pdf_poisson = np.exp(-s_grid)
    
    # Gráfico da Esquerda: Análise de Autovalores P(s)
    ax = axes[0]
    ax.hist(s_unfolded_log, bins=125, density=True, alpha=0.75, label='Dados P(s)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2.5, label='Teoria GOE')
    ax.set_title(f'a) Análise de Autovalores', fontsize=14)
    ax.set_xlabel('s (Espaçamento Normalizado)'); ax.set_ylabel('Densidade')
    ax.set_xlim(0, 4); ax.legend(loc='upper right')
    
    # Gráfico da Direita: Análise de Autovetores PR/N (VERSÃO MELHORADA)
    ax = axes[1]
    ax.hist(pr_values / N, bins=125, density=True, alpha=0.75, label='Dados PR/N', color='green')
    ax.axvline(1/3, color='purple', ls='--', lw=2.5, label=f'Teórico GOE ≈ {1/3:.4f}')
    
    # Adiciona uma linha para a média medida, para comparação visual
    ax.axvline(pr_n_mean, color='red', ls=':', lw=2, label=f'Média Medida = {pr_n_mean:.4f}')
    
    # APLICA O ZOOM: Define os limites do eixo X para focar na região do pico
    zoom_width = 0.05
    ax.set_xlim(1/3 - zoom_width, 1/3 + zoom_width)
    
    # TÍTULO MELHORADO: Inclui o valor médio medido
    ax.set_title(f'b) Análise de Autovetores (PR/N)', fontsize=14)
    ax.set_xlabel('PR / N');
    ax.legend(loc='upper right')
    
    fig.suptitle(f"Análise Espectral em X₀ ≈ 10^{log_X0:.0f}", fontsize=18, weight='bold')
    plt.show()

# --- Widget Interativo ---
interact(stratospheric_lab_final, 
         N=widgets.Dropdown(options=[1024, 2048, 4096], value=2048, description='N:'),
         log_X0=widgets.FloatSlider(min=8, max=28, step=1, value=22, description='X₀=10^', continuous_update=False),
         span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=4.0, description='Span:')
        );


interactive(children=(Dropdown(description='N:', index=1, options=(1024, 2048, 4096), value=2048), FloatSlider…

---

## 4. Resultados e Discussão: A Persistência da Assinatura

Os gráficos gerados na escala de $10^{22}$ revelam uma convergência quase perfeita com a teoria do Caos Quântico:

### Análise de Autovalores (Estatística de Espaçamento)

* **Média Medida $\langle r \rangle \approx 0.5337$**: Este valor está em erro ínfimo em relação ao valor teórico da GOE ($\approx 0.536$).
* **Distribuição $P(s)$**: O histograma azul segue a curva tracejada vermelha (Teoria GOE), demonstrando a **repulsão de níveis**. Ao contrário de um sistema aleatório (Poisson), os autovalores dos primos nesta escala evitam uns aos outros, mantendo uma rigidez estrutural.

### Análise de Autovetores (Localização e Ergodicidade)

* **Média Medida $\mathrm{PR}/N \approx 0.3337$**: A coincidência com o valor teórico de $1/3$ ($0.333\dots$) é absoluta.
* **Significado Físico**: Um $\mathrm{PR}/N$ de $1/3$ indica que os estados (autovetores) do Operador $M$ estão completamente estendidos (delocalizados). Isso demonstra que a informação da "tensão dos primos" não está concentrada em poucos números, mas distribuída de forma **ergódica** por todo o intervalo.

---

## 5. Conclusão Final do Experimento Estratosférico

A análise em $10^{22}$ encerra a fase experimental deste trabalho com uma constatação fundamental: **a ordem estatística dos números primos não é um fenômeno de "pequenos números"**.

1. **Robustez estatística**: A transição para a função $\mathrm{R}(x)$ de Riemann indica que a assinatura GOE permanece estável no regime observacional considerado, mesmo em escalas assintóticas extremas.
2. **Saturação da Evidência**: Ao atingirmos as mesmas escalas onde Odlyzko encontrou a GUE nos zeros, e encontrarmos a **GOE** na estrutura aritmética real, fechamos, no sentido operacional aqui definido, a análise do operador considerado.
3. **Evidência de Simetria**: A persistência do $\mathrm{PR}/N$ próximo a $1/3$ e de $\langle r \rangle$ próximo ao valor GOE em escalas astronômicas fornece evidência operacional robusta de que a simetria de espelhamento atua como um princípio estruturante no regime aritmético analisado.
